In [1]:
# Connecting directly to a sqlite database using python
#
# Objectives: 
#    Connect directly to a sqlite database
#    Retrieve query results as tuples
#    Connect to a sqlite database through pandas
#    Retrieve results as a pandas dataframe
#    Use pandas and sql in the same notebook 

In [3]:
import sqlite3

In [4]:
# to work directly with a sqlite database, we create a connection
# and a cursor to the connection
# you will use the cursor to send commands to and read results from the database

conn = sqlite3.connect("data/portal_mammals.sqlite")
c = conn.cursor()

In [6]:
# use the execute method to query the sqlite database
# use a loop to read the results back as a series of tuples

In [7]:
for row in c.execute('SELECT * FROM species LIMIT 5'):
    print(row)

('AB', 'Amphispiza', 'bilineata', 'Bird')
('AH', 'Ammospermophilus', 'harrisi', 'Rodent')
('AS', 'Ammodramus', 'savannarum', 'Bird')
('BA', 'Baiomys', 'taylori', 'Rodent')
('CB', 'Campylorhynchus', 'brunneicapillus', 'Bird')


In [11]:
# now that you can retrieve each row, you are able to access data within each row
# as a tuple. 
# here, we retrieve the first and third element for each row

In [10]:
for row in c.execute('SELECT * FROM species LIMIT 5'):
    print(row[0], row[2])

AB bilineata
AH harrisi
AS savannarum
BA taylori
CB brunneicapillus


In [15]:
# getting information about the sqlite database
# without a browser (such as db browser for sqlite), it's harder to get info about the
# tables and column names.
# we can use query the sqlite schema to all tables and data elements

In [17]:
for row in c.execute("SELECT * FROM sqlite_schema"):
    print(row)

('table', 'surveys', 'surveys', 2, 'CREATE TABLE surveys (\n\trecord_id BIGINT, \n\tmonth BIGINT, \n\tday BIGINT, \n\tyear BIGINT, \n\tplot_id BIGINT, \n\tspecies_id TEXT, \n\tsex TEXT, \n\thindfoot_length FLOAT, \n\tweight FLOAT\n)')
('table', 'species', 'species', 241, 'CREATE TABLE species (\n\tspecies_id TEXT, \n\tgenus TEXT, \n\tspecies TEXT, \n\ttaxa TEXT\n)')
('table', 'plots', 'plots', 242, 'CREATE TABLE plots (\n\tplot_id BIGINT, \n\tplot_type TEXT\n)')


In [ ]:
# to get detailed information about just one table, we can use pragma 

In [20]:
for row in c.execute('pragma table_info("surveys")'):
    print(row)

(0, 'record_id', 'BIGINT', 0, None, 0)
(1, 'month', 'BIGINT', 0, None, 0)
(2, 'day', 'BIGINT', 0, None, 0)
(3, 'year', 'BIGINT', 0, None, 0)
(4, 'plot_id', 'BIGINT', 0, None, 0)
(5, 'species_id', 'TEXT', 0, None, 0)
(6, 'sex', 'TEXT', 0, None, 0)
(7, 'hindfoot_length', 'FLOAT', 0, None, 0)
(8, 'weight', 'FLOAT', 0, None, 0)


In [ ]:
# working with sqlite and pandas dataframes
# 
# You may find it more convenient to read results directly into pandas instead of reading data row-by-row

In [21]:
import pandas as pd

In [31]:
# pandas has a built-in method "read_sql" that can populate a dataframe directly from a query
# you will need to pass this method a query and a connection to your sqlite database
# you can also (optionally) set an index

In [32]:
df_species = pd.read_sql("SELECT * FROM species", conn, index_col='species_id')

In [33]:
# the result is a pandas dataframe.

In [35]:
df_species.head()

,genus,species,taxa
species_id,,,
AB,Amphispiza,bilineata,Bird
AH,Ammospermophilus,harrisi,Rodent
AS,Ammodramus,savannarum,Bird
BA,Baiomys,taylori,Rodent
CB,Campylorhynchus,brunneicapillus,Bird


In [38]:
# using pandas and sql to analyze a dataset
# 
# you may find that some operations are much easier in sql than python or pandas, while others are 
# nearly impossible in SQL but very easy in pandas.
#
# For example, to calculate the average hindfoot length for each species in the surveys table
# we can create an aggregation table

In [51]:
# for longer queries, you may want to create a string to hold your sql and pass it as
# an argument.
query = """
SELECT species_id, AVG(hindfoot_length)
FROM surveys
WHERE hindfoot_length IS NOT NULL
GROUP BY species_id

"""

In [52]:
df_hf = pd.read_sql(query, conn)

In [53]:
df_hf

,species_id,AVG(hindfoot_length)
0,AH,33.000000
1,BA,13.000000
2,DM,35.982351
3,DO,35.607551
4,DS,49.948874
5,NL,32.294227
6,OL,20.532609
7,OT,20.267415
8,OX,19.125000
9,PB,26.115922


In [55]:
# what if you wanted the standard deviation, median, and intequartile range
# these calculations can be done in SQL, but you might find it much easier to use 
# python methods
#
# here, we'll use the describe() method in pandas
# first, we'll retrieve a dataset through SQL
# next, we'll generate summary statistics

In [56]:
query = "SELECT * FROM surveys"
df = pd.read_sql(query, conn)
df.describe()

,record_id,month,day,year,plot_id,hindfoot_length,weight
count,35549.000000,35549.000000,35549.000000,35549.000000,35549.000000,31438.000000,32283.000000
mean,17775.000000,6.477847,15.991195,1990.475231,11.397001,29.287932,42.672428
std,10262.256696,3.396925,8.257366,7.493355,6.799406,9.564759,36.631259
min,1.000000,1.000000,1.000000,1977.000000,1.000000,2.000000,4.000000
25%,8888.000000,4.000000,9.000000,1984.000000,5.000000,21.000000,20.000000
50%,17775.000000,6.000000,16.000000,1990.000000,11.000000,32.000000,37.000000
75%,26662.000000,10.000000,23.000000,1997.000000,17.000000,36.000000,48.000000
max,35549.000000,12.000000,31.000000,2002.000000,24.000000,70.000000,280.000000
